### Data & Innovation Day - Realtime - Azure AI Services and Fabric demonstration

The NOAA’s Center for Operational Oceanographic Products and Services (CO-OPS) provides real-time information on tides, water levels, currents, and other coastal oceanographic and meteorological data. This data serves a crucial purpose in aiding the maritime transportation industry by ensuring safe and efficient navigation through waterways and ports. In addition, current trends and predictions play a significant role in assisting individuals in preparing for potential flooding due to storms, tsunamis, and sea level fluctuations.

![A blue and black rectangle with a white cross  Description automatically generated](https://dataplatformblogcdn.azureedge.net/wp-content/uploads/2023/07/a-blue-and-black-rectangle-with-a-white-cross-des.png)

https://blog.fabric.microsoft.com/en-US/blog/microsoft-fabric-event-streams-generating-real-time-insights-with-python-kql-and-powerbi/

https://tidesandcurrents.noaa.gov/map/index.html


In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
%pip install azure-eventhub
%pip install azure-identity
%pip install aiohttp

StatementMeta(, 47381787-0f0a-4cf7-9b67-91de01da9d20, 10, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Not uninstalling typing-extensions at /home/trusted-service-user/cluster-env/trident_env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-564f305f-022b-43f9-b514-091f2ef21ae4
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0

**Collecting Coastal Water Level and Atmospheric Data with Python**

NOAA provides real-time water level information that is updated every 6 minutes. You can use HTTP Get API call to fetch water level and location information for a specific station. This API takes a StationID (full list is here) as a required input and some other optional parameters. 

```xml
<data>
<metadata id="8594900" name="Washington" lat="38.8733" lon="-77.0217"/>
<observations>
<wl t="2023-07-09 19:30" v="8.232" s="0.023" f="1,0,0,0" q="p"/>
</observations>
</data>
```

**Create a request payload, issue the HTTP call and retrieve the response from NOAA into a Python function**

In [12]:
import requests
import time
from datetime import datetime
# Fetch water level data from tidesandcurrents.noaa.gov
def fetch_water_level(station_id):
    url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
    # Collect water level for one station
    payload = {
        "product": "water_level",
        "application": "Fabric EventStream",  # Replace with your application name
        "datum": "STND",
        "station": station_id,  # Set station ID
        "time_zone": "gmt",
        "units": "english",  # Use "english" for feet, "metric" for meters
        "format": "json",
        "date": "latest",
    }
    response = requests.get(url, params=payload)
    if response.status_code == 200:
        data = response.json()
        return data
    return None

StatementMeta(, 47381787-0f0a-4cf7-9b67-91de01da9d20, 22, Finished, Available)

**Unit test of API Call**

```javascript
{'Current_time': '2024-01-12 08:20:25', 'Water Level Value': '8.937', 'Station Latitude': '47.5617', 'Station Longitude': '-122.6230', 'Station': 'Water Station'}
```

In [13]:
stationID = "9445958"  # Sample Bremerton, WA [9445958]
station_data = fetch_water_level(stationID)

if station_data is not None:
                # Fetch station location
                data = station_data["data"]
                if len(data) > 0:
                    water_level = data[0]["v"]
                station_latitude = station_data["metadata"]["lat"]
                station_longitude = station_data["metadata"]["lon"]
                # Create a data point with the time and value
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                # Generate the data structure
                water_point = {
                    "Current_time": current_time,
                    "Water Level Value": water_level,
                    "Station Latitude": station_latitude,
                    "Station Longitude": station_longitude,
                    "Station": "Water Station",
                }
print(water_point)


StatementMeta(, 47381787-0f0a-4cf7-9b67-91de01da9d20, 23, Finished, Available)

{'Current_time': '2024-04-29 13:06:16', 'Water Level Value': '20.404', 'Station Latitude': '47.5617', 'Station Longitude': '-122.6230', 'Station': 'Water Station'}


**Ingesting Data into Fabric Eventstreams (using Python)**

Now that we have a well-tested Python code to retrieve all the data we need from its source, let us see how we can package the same in a ‘EventData’ structure and send it on to a Fabric Eventstream. For now, we will focus on the Python code but in a subsequent, we will show how to connect the Python code to a Eventstream.

The above Python code acts as a Eventstream Producer/client and publishes water level data from NOAA every **6 minutes** to it. This is the interval at which NOAA refreshes its data. Refreshing at a higher frequency would be redundant.

In [14]:
# code for fabric event stream
EVENTSTREAM_CONNECTION_STR = "Endpoint=sb://esehamab2m01n7sux0kro6.servicebus.windows.net/;SharedAccessKeyName=key_c916bcd4-4274-938c-7928-9128f37bd34c;SharedAccessKey=sWPaynJsz3h4uP+D7zbLs88d5lTC+fxqG+AEhBaB2+A=;EntityPath=es_389f5a3e-d2d9-4936-9ab9-7a0629879502"
EVENTSTREAM_NAME = "es_389f5a3e-d2d9-4936-9ab9-7a0629879502"

StatementMeta(, 47381787-0f0a-4cf7-9b67-91de01da9d20, 24, Finished, Available)

In [15]:
import time
import json
from datetime import datetime
from azure.eventhub import EventData
from azure.eventhub.aio import EventHubProducerClient

async def run():
    """
    This function sends water level data to an EventStream using Azure Event Hubs.
    It fetches the current water level from a specified station and sends it as a JSON payload to the EventStream.
    The function runs indefinitely, generating a new data point every 6 minutes.
    """

    # Create a producer client to send messages to the EventStream.
    # Specify a connection string to your EventStream namespace and the eventstream name.
    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENTSTREAM_CONNECTION_STR, eventhub_name=EVENTSTREAM_NAME
    )

    # Specify the station ID
    stationID = "9445958"  # Sample Bremerton, WA [9445958]

    async with producer:
        while True:
            # Create a batch.
            event_data_batch = await producer.create_batch()

            # Fetch current water level in feet
            station_data = fetch_water_level(stationID)
           
            if station_data is not None:
                # Fetch station location
                data = station_data["data"]
                if len(data) > 0:
                    water_level = data[0]["v"]
                station_latitude = station_data["metadata"]["lat"]
                station_longitude = station_data["metadata"]["lon"]

                # Create a data point with the time and value
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                # Generate the data structure
                water_point = {
                    "Current_time": current_time,
                    "Water Level Value": water_level,
                    "Station Latitude": station_latitude,
                    "Station Longitude": station_longitude,
                    "Station": "Water Station",
                }

                # Convert the curve data to JSON format
                json_water_data = json.dumps(water_point)
                event_data_batch.add(EventData(json_water_data))

                # Send the batch of events to the event hub.
                await producer.send_batch(event_data_batch)

            # Wait for 6 minutes before generating the next point
            time.sleep(360)

await run()

StatementMeta(, 47381787-0f0a-4cf7-9b67-91de01da9d20, 25, Submitted, Running)